# Decision tree from scratch

Toy data set. Goal is to predict the type of fruit.

In [3]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [4]:
header = ["color", "diameter", "label"]

You may notice, the data is not perfectly seperable. Meaning that the Apple and Lemon can't be distinguished from each other since the values of their features are equivalent.

### CART (Classification and Regression Trees)



- We create a **root node**. It represents the entire population or sample and this further gets divided into two or more homogeneous sets
- This dividing process is called **splitting**.
- Further sub-nodes are called **Decision nodes**.
- **Leaf/Terminal nodes** are nodes that do not split.
- The sub-node is always called the **Child** node of the original **Parent node**
- **Pruning** is when we remove sub-nodes of a decision node. It's the opposite of splitting. 

### The trick to getting a good CART

The goal is to un-mix the labels as purely as possible. In order to do so, one must ask the right questions at the right time. 

To do that, we need to quantify how much a questions helps to unmix the labels. We can quantify how much impurity there is in a single node using ***Gini impurity***

We can quantify how much a question reduces the impurity (label uncertainty) using a concept called ***Information Gain***.

### What question to ask?

Possible questions
- is the color green?
- is the diameter >= 3? 
- is the color yellow?
- et cetera

## Some code

A **set** is a type of data that stores a set of unique things. It rejects any duplicate elements.

Example:

In [5]:
a = set()
a

set()

This just represents an empty set for now.

In [6]:
a.add(1)
a

{1}

This represents with only 1 inside the set.

Here we use set to return unique values for a certain column in a dataset. That column is up to you to choose.

In [7]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset.""" 
    return set([row[col] for row in rows])

Example:

In [8]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

This function counts the number of each type of example in a dataset.

In [9]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

Example:

In [10]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

Test if a value is numeric using the isinstance method.

In [11]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

repr() vs str(). 

- The goal of repr() is to be unambiguous.
- The goal of str() is to be printable.

Unambiguous means being able to pass it to **eval()**. 
That means basically that it must be a valid Python command that can be run in the interpreter.

In [12]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [13]:
q = Question(0, 'Green')
q

Is color == Green?

In response to a question, we divide or ***partition*** the data into two subsets.
- The first contains all False
- The second contains all True

In [14]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [15]:

#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [16]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

### Quantifying uncertainty


impurty = chance of being incorrect if you randomly assign a label to an example in the same set. So if we randomly assign a label to an example. The odds of getting it right is the purity.


How we will do it:

- we begin by calculating the uncertainty of our starting set.
- for each question we can ask, we try partitioning the data and calculate the uncertainty of the resulting child nodes. *We take the weighted average of both child nodes*
- then, we subtract this from our starting uncertainty and that will be our **information gain**
- as we go, we keep track of the question that gives the most gain. That will be the question we ask at this node. 


### Let's see how this looks in code:

- input = the dataset which is a bunch of rows.
- counts = counts the number of each type of an example in the form of a **dictionairy**. E.g., {'Apple': 2, 'Grape': 4}.
- impurity = becausean impurity score ranges from 0 to 1, 1 will be our starting-off point. From there it will be subtracted.

In [20]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    
    # for all elements in the dictionary...
    for lbl in counts:
        # each key (label) is divided by the number of instancers in the dataset.
        prob_of_lbl = counts[lbl] / float(len(rows))
        # impurity = probability of label to the power of 2. 
        impurity -= prob_of_lbl**2
    return impurity

In [21]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [22]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [24]:
gini(training_data)

0.6399999999999999

In [25]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)

0.7999999999999998

In [28]:
yoyo = [['Apple'], ['Apple'], ['Grape'], ['Grape'], ['Lemon']]
gini(yoyo)

0.6399999999999999

Moving on; Information Gain.



In [35]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    
    # return a weighted average from both left and right ginis
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [31]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

Info gain takes the arguments:
- true_rows (resulting from the partition algorithm)
- false_rows (resulting from the partition algorithm)
- current_uncertainty a.k.a. mother node (resulting from the gini algorithm)

In [32]:
# If we partitioned using Green
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [33]:
# If we partitioned using Red
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

It looks like the red split is better than the green split. 

### algorithm to find the best splits.

In [36]:
def find_best_split(rows):
    """
    Find the best question to ask by iterating over every feature / value
    and calculating the information gain.
    """
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows) # uncertainty in the mother node
    n_features = len(rows[0]) - 1  # number of columns

    
    
    for col in range(n_features): # iterate over all features values  

        values = set([row[col] for row in rows])  # set is used to produce a dictionary of only UNIQUE values

        for val in values:  # for each value in the unique value dictionaire...

            question = Question(col, val) # for each value in the unique value dictionaire we generate a question

            # use the question to partition the data
            true_rows, false_rows = partition(rows, question)

            # Discard any questions that fail to produce a split
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [37]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

### Putting it all together using Recursion.

In [38]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [39]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [40]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [41]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [42]:
my_tree = build_tree(training_data)

In [44]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}
